In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import cred

import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import cosine
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_recently_played_df():
    scope = "user-read-recently-played"

    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred.CLIENT_ID
                                                   , client_secret= cred.CLIENT_SECRET
                                                   , redirect_uri=cred.REDIRECT_URL
                                                   , scope=scope))

    resultsDf = pd.DataFrame(columns=['Title', 'Attribute', 'Value'])

    results = sp.current_user_recently_played()
    for idx, item in enumerate(results['items']):
        track = item['track']
        track_id = track['id']
        features = sp.audio_features(track_id)
        data_list = []
        for att in ['speechiness', 'acousticness', 'instrumentalness', 'tempo', 'energy']:
            # print(track['name'], ': ' ,f'{att} - ', features[0][att])
            data_to_append = pd.DataFrame([{'Title':f'{track["name"]}', 'Attribute':f'{att}', 'Value':f'{features[0][att]}'}])
            resultsDf = pd.concat([resultsDf,data_to_append])

    resultsDf = resultsDf.drop_duplicates()
    resultsDf = pd.pivot(resultsDf, index='Title', columns='Attribute', values='Value')
    resultsDf = resultsDf.reset_index()
    resultsDf = resultsDf.rename(columns={"Attribute": "index", 'Title':'title'})
    resultsDf = resultsDf.rename_axis(None, axis = 1)

    return resultsDf
    # print('\n', track.keys())

In [3]:
def get_specific_playlist_df(playlist):
    scope = "user-library-read"

    sp = spotipy.Spotify(auth_manager = SpotifyOAuth(client_id = cred.CLIENT_ID
                                                    , client_secret = cred.CLIENT_SECRET
                                                    , redirect_uri = cred.REDIRECT_URL
                                                    , scope = scope))

    resultsDf = pd.DataFrame(columns=['Title', 'ID', 'Attribute', 'Value'])

    full_playlist_list = sp.current_user_playlists()['items']

    for i in range(len(full_playlist_list)):
        if full_playlist_list[i]['name']==playlist:
            playlist_id = full_playlist_list[i]['id']
            no_of_songs = full_playlist_list[i]['tracks']['total']
        else:
            pass
    
    start = 0
    end = 100
    counter=0
    if start < no_of_songs:
        for x in range(start, end):
            if end==100:
                songs_in_playlist = sp.playlist_items(playlist_id, offset=start)['items']
            else:
                songs_in_playlist.extend(sp.playlist_items(playlist_id,offset=start)['items'])
                pass
            start+=100
            end+=100
            counter+=1

        for i in range(len(songs_in_playlist)):
            track = songs_in_playlist[i]['track']
            track_id = track['id']
            features = sp.audio_features(track_id)

            for att in ['speechiness', 'acousticness', 'instrumentalness', 'tempo', 'energy']:
                data_to_append = pd.DataFrame([{'Title':f'{track["name"]}','ID':f'{track["id"]}', 'Attribute':f'{att}', 'Value':f'{features[0][att]}'}])
                resultsDf = pd.concat([resultsDf,data_to_append])

    resultsDf = resultsDf.drop_duplicates()
    resultsDf = resultsDf.reset_index().pivot(index=['Title', 'ID'], columns='Attribute', values='Value')
    resultsDf = resultsDf.reset_index()
    resultsDf = resultsDf.rename(columns={"Attribute": "index", 'Title':'title'})
    resultsDf = resultsDf.rename_axis(None, axis = 1)

    # resultsDf_features = resultsDf[['speechiness', 'acousticness', 'instrumentalness', 'tempo', 'energy']]

    return resultsDf

# get_specific_playlist_df('country')

In [4]:
# recently_played_df = get_recently_played_df()
# playlist_df = get_specific_playlist_df(playlist='country')

In [5]:
def process_playlist_data(recently_played_df, playlist_df):
    recently_played_list = recently_played_df['title'].unique().tolist()
    
    playlist_df_filtered = playlist_df[~playlist_df['title'].isin(recently_played_list)]
    playlist_df_filtered = playlist_df_filtered.rename_axis(None, axis = 1)

    playlist_df_filtered_features = playlist_df_filtered[['speechiness', 'acousticness', 'instrumentalness', 'tempo', 'energy']]
    return playlist_df_filtered, playlist_df_filtered_features

# playlist_df_filtered, playlist_df_filtered_features = process_playlist_data(recently_played_df, playlist_df)

# Develop The Recommendations

In [15]:
def order_by_similarity(playlist_features_df,recently_played_df,weight_factor,feature_of_importance):
    for col in playlist_features_df.columns.tolist():
        playlist_features_df[col] = playlist_features_df[col].astype(float)

    playlist_features_df['weight'] = playlist_features_df[feature_of_importance].apply(lambda x: weight_factor ** (-x))
    playlist_features_df_weighted = playlist_features_df.copy()
    playlist_features_df_weighted.update(playlist_features_df_weighted.iloc[:,:-1].mul(playlist_features_df_weighted.weight.astype(float),0))  
    playlist_features_df_weighted_final = playlist_features_df_weighted.iloc[:, :-1]
    playlist_df_filtered_vector = playlist_features_df_weighted_final.sum(axis = 0)

    # print(playlist_df_filtered_vector)

    print(recently_played_df.drop(['title'], axis = 1).values)

    recently_played_df['sim'] = cosine_similarity(recently_played_df.values, playlist_df_filtered_vector.values.reshape(2, -1))[:,0]
    recently_played_df = recently_played_df.rename_axis(None, axis = 1)
    return recently_played_df.sort_values(by = 'sim', ascending=False)


In [7]:
# outputDf = order_by_similarity(playlist_features_df = playlist_df_filtered_features
#                     , recently_played_df=recently_played_df
#                     , weight_factor = 1.2
#                     , feature_of_importance = 'acousticness')

In [16]:
def execute_script(playlist, weight_factor):
    # get data
    recently_played_df = get_recently_played_df()
    playlist_df = get_specific_playlist_df(playlist=playlist)

    # process data further
    playlist_df_filtered, playlist_df_filtered_features = process_playlist_data(recently_played_df, playlist_df)

    # output
    outputDf_acousticness = order_by_similarity(playlist_features_df = playlist_df_filtered_features
                    , recently_played_df=recently_played_df
                    , weight_factor = weight_factor
                    , feature_of_importance = 'acousticness')

    outputDf_speechiness = order_by_similarity(playlist_features_df = playlist_df_filtered_features
                    , recently_played_df=recently_played_df
                    , weight_factor = weight_factor
                    , feature_of_importance = 'speechiness')
    
    return outputDf_acousticness, outputDf_speechiness

In [17]:
execute_script(playlist = 'country', weight_factor = 1.2)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=7B2ETOSXeXPfWQl4gLjcat:
 Max Retries, reason: too many 429 error responses

In [14]:
scope = "playlist-read-private playlist-read-collaborative playlist-modify-public playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred.CLIENT_ID
                                                   , client_secret= cred.CLIENT_SECRET
                                                   , redirect_uri=cred.REDIRECT_URL
                                                   , scope=scope))

In [15]:
playlists = sp.current_user_playlists()
playlist_info = [(playlist['name'], playlist['id']) for playlist in playlists['items']]
playlist_info

[('Discover Weekly', '37i9dQZEVXcCatZ0bLrVoY'),
 ('Race Day', '4I7Q3d9jocxZ61VkAfAY7X'),
 ('#bangersoftheweek', '1pJU69B3ZLMCParxhVgcI9'),
 ('Alternative ', '3s5NbMIkm5dWDSSjvOzLk1'),
 ('Soft Rock', '37i9dQZF1DX6xOPeSOGone'),
 ('Rock', '3nF3jQ4cM6Usa9ImKPDNjY'),
 ('country', '0sb9jCAEePyFjPQphglYSa'),
 ('rap', '0SNZFINx5LGxsQX8OWFV0F'),
 ('Other', '4jIM3wc2sLn0kOLDCQE9ss'),
 ('Ausländer Lieder', '3yM2MPQRahLuWaMj6DbE9Y'),
 ('alternative stuff', '7ss6ti7RQn5qpth2W6hKB4'),
 ('New Music Friday', '37i9dQZF1DX4JAvHpjipBk'),
 ('Fresh Finds', '37i9dQZF1DWWjGdmeTyeJ6'),
 ('German School of Madison Falken', '2itEKE96RREghEcDlezEL4')]

In [18]:
# def get_playlist_names_and_ids(sp):
#     playlists = sp.current_user_playlists()
#     playlist_info = [(playlist['name'], playlist['id']) for playlist in playlists['items']]
#     return playlist_info

# def get_playlist_id_by_name(sp, playlist_name):
#     playlists_info = get_playlist_names_and_ids(sp)
#     for name, id in playlists_info:
#         if name == playlist_name:
#             return id
#     return "Playlist not found!!!"


# discover_weekly_id = get_playlist_id_by_name(sp, "Discover Weekly")


def get_playlist_id_by_name(sp, playlist_name):
    playlists = sp.current_user_playlists()
    for playlist in playlists['items']:
        if playlist['name'] == playlist_name:
            return playlist['id']
    return None

discover_weekly_id = get_playlist_id_by_name(sp, "Discover Weekly")
chosen_playlist_id = get_playlist_id_by_name(sp, "country")

In [20]:
def get_tracks_from_playlist(sp, playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = []
    for item in results['items']:
        track = item['track']
        tracks.append((track['id'], track['name'], [artist['name'] for artist in track['artists']]))
    return tracks

discover_weekly_tracks = get_tracks_from_playlist(sp, discover_weekly_id)
chosen_playlist_tracks = get_tracks_from_playlist(sp, chosen_playlist_id)

In [23]:
discover_weekly_tracks

[('7CcEZJUdwBVjwwPe0pHqLz',
  'The Devil Never Sleeps ( Fiddle)',
  ['The Ghost of Johnny Cash', 'Philip Bowen']),
 ('60xGURAbB9FPslZn4I3RoH', 'Daughter of a King', ['Trailer Flowers']),
 ('3yz3RIqptKAVjsKbsPWaga', 'Wasted in Athens', ['The Ocho']),
 ('0IM9MLjcfbVpXmoi2A2pug', 'Does He Make You Happy? - Demo', ['Tors']),
 ('2cxMeI8J7GWAiBZZYAf7ig', 'Live Your Life', ['Dirty Heads']),
 ('5JYdC5yOeoudrUt3AfB4pZ', 'Ella', ['Darren Kiely']),
 ('1R4dOKtPzW5NnFsQ7n7DwF', 'Fighting Tears', ['Wade Forster']),
 ('4dyEaJfH5Yurs8Eycd6VpC', 'Too Drunk To Drive', ['Jake Puliti']),
 ('1JGY9xbWux7sogcucrz5TS', 'Straight To Hell', ['Drivin N Cryin']),
 ('48sgSBnvpO1hhEyz4ZyTnd', 'Appalachian Woman', ['Caledonia']),
 ('2jyXJ3RB21d3tZVZRiHtKm', 'Long Way to Go', ['P!nk', 'The Lumineers']),
 ('7H3EzoWKPRcagDVhJU4WxI', 'Outskirts of Heaven', ['Matt Schuster']),
 ('7EuqJ4pd36v3li7aBxLH85', 'Younger by the Rye', ['The Last Knife Fighter']),
 ('26HQt3tNOagBa4Av5lftP2', 'Sundown', ['Nic D', 'Vwillz']),
 ('0rZ

In [ ]:
def get_audio_features(sp, track_ids):
    audio_features = sp.audio_features(track_ids)
    return audio_features

# Usage
# Assuming you have a list of track IDs, you can call the function like this:
track_ids = ["track_id_1", "track_id_2", "track_id_3"]  # Replace with your actual track IDs
audio_features = get_audio_features(sp, track_ids)

# `audio_features` will contain a list of dictionaries, where each dictionary represents the audio features of a track.
for features in audio_features:
    if features:
        print(f"Track ID: {features['id']}")
        print(f"Acousticness: {features['acousticness']}")
        print(f"Danceability: {features['danceability']}")
        # Add more feature fields as needed
        print()

In [22]:
def calculate_cosine_similarity(track1, track2):
    features1 = np.array(track1)  # Replace with the actual features you want to use
    features2 = np.array(track2)  # Replace with the actual features you want to use
    return 1 - cosine(features1, features2)

similarity_scores = []
for dw_track in discover_weekly_tracks:
    for chosen_track in chosen_playlist_tracks:
        similarity = calculate_cosine_similarity(dw_track, chosen_track)
        similarity_scores.append((dw_track, chosen_track, similarity))

# Sort the similarity scores by descending order
similarity_scores.sort(key=lambda x: x[2], reverse=True)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [32]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import numpy as np
from scipy.spatial.distance import cosine
import cred
import pandas as pd
import numpy as np

scope = "playlist-read-private playlist-read-collaborative playlist-modify-public playlist-modify-private"

def get_playlist_names_and_ids(sp):
    playlists = sp.current_user_playlists()
    playlist_info = [(playlist['name'], playlist['id']) for playlist in playlists['items']]
    return playlist_info

def get_playlist_id_by_name(sp, playlist_name):
    playlists_info = get_playlist_names_and_ids(sp)
    for name, id in playlists_info:
        if name == playlist_name:
            return id
    return None

def get_audio_features(sp, track_ids):
    audio_features = sp.audio_features(track_ids)
    return audio_features

def calculate_cosine_similarity(features1, features2):
    return 1 - cosine(features1, features2)

def create_new_playlist(sp, playlist_name):
    user_id = sp.me()['id']  # Get your Spotify user ID
    playlist = sp.user_playlist_create(user_id, playlist_name)
    return playlist['id']

def add_tracks_to_playlist(sp, playlist_id, track_uris):
    sp.playlist_add_items(playlist_id, track_uris)

def main():
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred.CLIENT_ID
                                                   , client_secret= cred.CLIENT_SECRET
                                                   , redirect_uri=cred.REDIRECT_URL
                                                   , scope=scope))

    # Replace with your chosen playlist name
    chosen_playlist_name = "country"
    discover_weekly_name = "Discover Weekly"
    new_playlist_name = "Sorted Discover Weekly"  # Choose a name for the new playlist

    # Get playlist IDs
    chosen_playlist_id = get_playlist_id_by_name(sp, chosen_playlist_name)
    discover_weekly_id = get_playlist_id_by_name(sp, discover_weekly_name)

    if not chosen_playlist_id or not discover_weekly_id:
        print("Playlist not found.")
        return

    # Get track IDs from the chosen playlist
    chosen_playlist_tracks = get_tracks_from_playlist(sp, chosen_playlist_id)

    # Get track IDs from Discover Weekly
    discover_weekly_tracks = get_tracks_from_playlist(sp, discover_weekly_id)

    # Extract audio features for chosen playlist tracks
    chosen_playlist_track_ids = [track[0] for track in chosen_playlist_tracks]
    chosen_playlist_audio_features = get_audio_features(sp, chosen_playlist_track_ids)

    # Create a DataFrame to store track names, artist names, and similarity scores
    track_data = []

    # Loop through Discover Weekly tracks and calculate cosine similarity
    for dw_track in discover_weekly_tracks:
        dw_track_id = dw_track[0]
        dw_track_name = dw_track[1]
        dw_track_artists = ', '.join(dw_track[2])  # Join multiple artists with a comma
        dw_track_audio_features = get_audio_features(sp, [dw_track_id])[0]

        if dw_track_audio_features:
            similarity = calculate_cosine_similarity(
                np.array([dw_track_audio_features['acousticness']
                        , dw_track_audio_features['danceability']
                        , dw_track_audio_features['energy']
                        , dw_track_audio_features['valence']
                        , dw_track_audio_features['instrumentalness']
                        , dw_track_audio_features['liveness']
                        , dw_track_audio_features['loudness']
                        , dw_track_audio_features['speechiness']
                        , dw_track_audio_features['tempo']
                        , dw_track_audio_features['key']]),  # Add more features here
                np.array([chosen_playlist_audio_features[0]['acousticness']
                        , chosen_playlist_audio_features[0]['danceability']
                        , chosen_playlist_audio_features[0]['energy']
                        , chosen_playlist_audio_features[0]['valence']
                        , chosen_playlist_audio_features[0]['instrumentalness']
                        , chosen_playlist_audio_features[0]['liveness']
                        , chosen_playlist_audio_features[0]['loudness']
                        , chosen_playlist_audio_features[0]['speechiness']
                        , chosen_playlist_audio_features[0]['tempo']
                        , chosen_playlist_audio_features[0]['key']])  # Replace with chosen playlist track's features
            )
            track_data.append((dw_track_name, dw_track_artists, similarity, dw_track_id))  # Store track name, artist names, similarity score, and track ID

    # Create a DataFrame from the track data
    df = pd.DataFrame(track_data, columns=['Track Name', 'Artist Names', 'Similarity Score', 'Track ID'])

    # Sort the DataFrame by similarity score in descending order
    df = df.sort_values(by='Similarity Score', ascending=False)

    # Print the sorted DataFrame
    display(df)

    # Return the track IDs of the sorted tracks
    sorted_track_ids = df['Track ID'].tolist()

    # Create a new playlist and add the sorted tracks to it
    new_playlist_id = create_new_playlist(sp, new_playlist_name)

    # Clear the existing tracks in the Discover Weekly playlist
    if new_playlist_id:
        sp.playlist_remove_all_occurrences_of_items(new_playlist_id, sorted_track_ids)

    add_tracks_to_playlist(sp, new_playlist_id, sorted_track_ids)

    print(f"New playlist '{new_playlist_name}' created with sorted tracks.")

if __name__ == "__main__":
    main()

,Track Name,Artist Names,Similarity Score,Track ID
3,Does He Make You Happy? - Demo,Tors,0.999969,0IM9MLjcfbVpXmoi2A2pug
16,Where Did You Go I'm Less of a Mess These Days,Blue October,0.999925,4LawS8eOxHBRPrdEfBm1Qq
21,Wargasm [Feat. RMR],"Billy Strings, RMR",0.999881,6oOIAmsQn4uG2Z9HO1agxJ
27,Alright,The Rural Alberta Advantage,0.999854,4gT4dOMCCTEHevIFGUNtWb
28,Georgia Law Man,Poor Man's Poison,0.999723,09LPOZPkdbTYY8TGHGsrb7
24,First Cigarette,Travis Meadows,0.999621,12WPyTmExymjR4LW5wPZj9
14,Love Is Blind,Nate Smith,0.999602,0rZxDClq3zo0rvPOShiYzi
6,Fighting Tears,Wade Forster,0.999590,1R4dOKtPzW5NnFsQ7n7DwF
4,Live Your Life,Dirty Heads,0.999585,2cxMeI8J7GWAiBZZYAf7ig
9,Appalachian Woman,Caledonia,0.999316,48sgSBnvpO1hhEyz4ZyTnd


New playlist 'Sorted Discover Weekly' created with sorted tracks.
